In [1]:
#Problem: Predicts if stock price of dow jones will increase or decrease 
#using top news headlines

In [2]:
#import the necessary libraries
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [3]:
#load and store the datasets
df_reddit = pd.read_csv(r"C:\Users\kianh\finsights\finsights_app\finsights_dow_dataset\RedditNews.csv")
df_price = pd.read_csv(r"C:\Users\kianh\finsights\finsights_app\finsights_dow_dataset\upload_DJIA_table.csv")
df_cn = pd.read_csv(r"C:\Users\kianh\finsights\finsights_app\finsights_dow_dataset\Combined_News_DJIA.csv")

In [4]:
df_price.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [5]:
df_price.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [6]:
#get the merged data set
#merges combined_news and the price data set
df_merge = df_cn.merge(df_price,how='outer', on='Date')

In [7]:
df_merge.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Taliban wages war on humanitarian aid workers',"b'Russia: World ""can forget about"" Georgia\'s...",b'Darfur rebels accuse Sudan of mounting major...,b'Philippines : Peace Advocate say Muslims nee...,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688


In [8]:
#combine the top news headlines
headlines = []
for row in range(0, len(df_merge.index)):
    headlines.append(' '.join(str(x) for x in df_merge.iloc[row, 2:27]) )

In [9]:
#clean the data(remove b' and b")
clean_headlines = []
for i in range(0, len(headlines)):
    clean_headlines.append(re.sub("b[(')]", '', headlines[i]))
    clean_headlines[i] = re.sub('b[(")]', '',clean_headlines[i])
    clean_headlines[i] = re.sub("\'", '', clean_headlines[i])

In [10]:
pip install datetime

Note: you may need to restart the kernel to use updated packages.


In [11]:
from datetime import datetime
user_date = '2023-07-21'
user_date

'2023-07-21'

In [12]:
df_merge.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top22,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Taliban wages war on humanitarian aid workers',"b'Russia: World ""can forget about"" Georgia\'s...",b'Darfur rebels accuse Sudan of mounting major...,b'Philippines : Peace Advocate say Muslims nee...,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688


In [13]:
#add the clean headlines to the merge data set
df_merge['Combined_News'] = clean_headlines
df_merge['Combined_News'][0]

'Georgia downs two Russian warplanes as countries move to brink of war" BREAKING: Musharraf to be impeached. Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube) Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire Afghan children raped with impunity, U.N. official says - this is sick, a three year old was raped and they do nothing" 150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets. Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SOs side" The enemy combatent trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO] Did the U.S. Prep Georgia for War with Russia? Rice Gives Green Light for Israel to Attack Iran: S

In [14]:
#Polarity = from -1 to 1 to see how positive or negative the statement is
#subjectibity = 0 t 1 on how objective the news article is
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [15]:
#Create 2 new columns
df_merge['Subjectivity'] = df_merge['Combined_News'].apply(getSubjectivity)
df_merge['Polarity'] = df_merge['Combined_News'].apply(getPolarity)

In [16]:
df_merge.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top25,Open,High,Low,Close,Volume,Adj Close,Combined_News,Subjectivity,Polarity
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,"b""No Help for Mexico's Kidnapping Surge""",11432.089844,11759.959961,11388.040039,11734.320312,212830000,11734.320312,Georgia downs two Russian warplanes as countri...,0.267549,-0.048568
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,"b""So this is what it's come to: trading sex fo...",11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,Why wont America and Nato help us? If they won...,0.374806,0.121956
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,"b""BBC NEWS | Asia-Pacific | Extinction 'by man...",11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,Remember that adorable 9-year-old who sang at ...,0.536234,-0.044302
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,U.S. refuses Israel weapons to attack Iran: r...,0.364021,0.011398
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,b'Philippines : Peace Advocate say Muslims nee...,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,All the experts admit that we should legalise ...,0.375099,0.040677


In [17]:
#Get the sentiment score
#Create a function to get the sentiment scores
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [18]:
#get the sentiment scores for each day
#make this a function that I can use for just one piece of text
compound = []
neg = []
pos = []
neutral = []
SIA = 0

for i in range(0, len(df_merge['Combined_News'])):
    SIA = getSIA(df_merge['Combined_News'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neutral.append(SIA['neu'])
    pos.append(SIA['pos'])

In [19]:
#Store the sentiment scores in the merge data frame
df_merge['Compound'] = compound
df_merge['Negative'] = neg
df_merge['Neutral'] = neutral
df_merge['Positive'] = pos

In [20]:
#create a list of columns to keep
keep_cols = ['Open', 'High', 'Close', 'Low', 'Volume', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive', 'Label']
#keep_cols is not in df_comb, so I need to fix this
df_merge = df_merge[keep_cols]

In [21]:
df_merge.head()

,Open,High,Close,Low,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,11432.089844,11759.959961,11734.320312,11388.040039,212830000,0.267549,-0.048568,-0.9982,0.235,0.724,0.041,0
1,11729.669922,11867.110352,11782.349609,11675.530273,183190000,0.374806,0.121956,-0.9858,0.191,0.721,0.089,1
2,11781.700195,11782.349609,11642.469727,11601.519531,173590000,0.536234,-0.044302,-0.9715,0.128,0.816,0.056,0
3,11632.809570,11633.780273,11532.959961,11453.339844,182550000,0.364021,0.011398,-0.9809,0.146,0.788,0.066,0
4,11532.070312,11718.280273,11615.929688,11450.889648,159790000,0.375099,0.040677,-0.9882,0.189,0.717,0.094,1


In [22]:
df_cn.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [23]:
#Calculate the daily percentage change
#Daily percentage change = (CloseDay1 - CloseDay2) / CloseDay1
def calculate_daily_percentage_change(df, price_column='Close'):
    """
    Calculate the daily percentage change in stock prices and add it as a new column in the DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock price data with a 'Date' column and a column for the 'Close' price.
        price_column (str, optional): The column name that contains the stock price data. Default is 'Close'.

    Returns:
        pd.DataFrame: DataFrame with an additional 'Daily_Pct_Change' column representing the daily percentage change in stock prices.
    """
    # Sort the DataFrame by date to ensure it is in chronological order
    #df = df.sort_values(by='Date', ascending=True)

    # Calculate the daily percentage change
    df.loc[:, 'Daily_Pct_Change'] = df.loc[:, price_column].pct_change() * 100

    # Drop the NaN value in the first row (resulting from the calculation of percentage change)
    df = df.dropna()

    return df

In [36]:
df_merge['Daily_Pct_Change'] = (df_merge['Close'].pct_change() * 100)
df_merge.dropna()

,Open,High,Close,Low,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label,Daily_Pct_Change
1,11729.669922,11867.110352,11782.349609,11675.530273,183190000,0.374806,0.121956,-0.9858,0.191,0.721,0.089,1,0.409306
2,11781.700195,11782.349609,11642.469727,11601.519531,173590000,0.536234,-0.044302,-0.9715,0.128,0.816,0.056,0,-1.187199
3,11632.809570,11633.780273,11532.959961,11453.339844,182550000,0.364021,0.011398,-0.9809,0.146,0.788,0.066,0,-0.940606
4,11532.070312,11718.280273,11615.929688,11450.889648,159790000,0.375099,0.040677,-0.9882,0.189,0.717,0.094,1,0.719414
5,11611.209961,11709.889648,11659.900391,11599.730469,215040000,0.457692,0.047756,-0.9892,0.177,0.753,0.070,1,0.378538
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,17355.210938,17355.210938,17140.240234,17063.080078,138740000,0.352311,-0.014015,-0.9644,0.148,0.758,0.094,0,-1.497118
1985,17190.509766,17409.720703,17409.720703,17190.509766,112190000,0.352649,0.046560,-0.9571,0.132,0.767,0.102,1,1.572209
1986,17456.019531,17704.509766,17694.679688,17456.019531,106380000,0.389617,0.052622,-0.9975,0.225,0.684,0.091,1,1.636781
1987,17712.759766,17930.609375,17929.990234,17711.800781,133030000,0.382566,0.011243,-0.9977,0.202,0.738,0.061,1,1.329838


In [38]:
len(df_merge)

1989

Open            1.142679e+04
High            1.163213e+04
Close           1.162806e+04
Low             1.142679e+04
Volume          1.387900e+08
Subjectivity    3.087121e-01
Polarity        8.011364e-02
Compound       -9.760000e-01
Negative        1.450000e-01
Neutral         7.780000e-01
Positive        7.800000e-02
Label           1.000000e+00
Name: 10, dtype: float64

,Open,High,Close,Low,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,11432.089844,11759.959961,11734.320312,11388.040039,212830000,0.267549,-0.048568,-0.9982,0.235,0.724,0.041,0
1,11729.669922,11867.110352,11782.349609,11675.530273,183190000,0.374806,0.121956,-0.9858,0.191,0.721,0.089,1
2,11781.700195,11782.349609,11642.469727,11601.519531,173590000,0.536234,-0.044302,-0.9715,0.128,0.816,0.056,0
3,11632.809570,11633.780273,11532.959961,11453.339844,182550000,0.364021,0.011398,-0.9809,0.146,0.788,0.066,0
4,11532.070312,11718.280273,11615.929688,11450.889648,159790000,0.375099,0.040677,-0.9882,0.189,0.717,0.094,1


,Open,High,Close,Low,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,11432.089844,11759.959961,11734.320312,11388.040039,212830000,0.267549,-0.048568,-0.9982,0.235,0.724,0.041,0
1,11729.669922,11867.110352,11782.349609,11675.530273,183190000,0.374806,0.121956,-0.9858,0.191,0.721,0.089,1
2,11781.700195,11782.349609,11642.469727,11601.519531,173590000,0.536234,-0.044302,-0.9715,0.128,0.816,0.056,0
3,11632.809570,11633.780273,11532.959961,11453.339844,182550000,0.364021,0.011398,-0.9809,0.146,0.788,0.066,0
4,11532.070312,11718.280273,11615.929688,11450.889648,159790000,0.375099,0.040677,-0.9882,0.189,0.717,0.094,1


Open            9.626420e+03
High            9.713710e+03
Close           9.683410e+03
Low             9.580930e+03
Volume          2.240300e+08
Subjectivity    2.613761e-01
Polarity        5.049137e-02
Compound        7.817000e-01
Negative        8.500000e-02
Neutral         8.110000e-01
Positive        1.040000e-01
Label           1.000000e+00
Name: 277, dtype: float64

Open            9.626420e+03
High            9.713710e+03
Close           9.683410e+03
Low             9.580930e+03
Volume          2.240300e+08
Subjectivity    2.613761e-01
Polarity        5.049137e-02
Compound        7.817000e-01
Negative        8.500000e-02
Neutral         8.110000e-01
Positive        1.040000e-01
Label           1.000000e+00
Name: 277, dtype: float64

In [31]:
# #issue is that df_subset only has one value
#after the first nan value, the loop stops and add values for Daily_Pct_Change in df_merge
# for i in range(1, len(df_merge)):
#     df_subset = df_merge.iloc[:i + 1]  # Subset of the DataFrame up to the current row
#     df_subset = calculate_daily_percentage_change(df_subset)
#     if not pd.isna(df_merge.loc[i, 'Daily_Pct_Change']):
#         df_merge.loc[i, 'Daily_Pct_Change'] = df_subset.loc[i, 'Daily_Pct_Change']

C:\Users\kianh\AppData\Local\Temp\ipykernel_19284\58553539.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Daily_Pct_Change'] = df.loc[:, price_column].pct_change() * 100


KeyError: 'Daily_Pct_Change'

Open                1.061177e+04
High                1.064495e+04
Close               1.062469e+04
Low                 1.059484e+04
Volume              1.661400e+08
Subjectivity        4.084784e-01
Polarity            2.711474e-02
Compound           -9.920000e-01
Negative            1.350000e-01
Neutral             8.040000e-01
Positive            6.000000e-02
Label               1.000000e+00
Daily_Pct_Change    1.210967e-01
Name: 400, dtype: float64

In [39]:
len(df_merge)

1989

In [41]:
#create a list of columns to keep
keep_cols = ['Open', 'High', 'Close', 'Low', 'Volume', 'Subjectivity', 'Polarity', 'Compound', 'Negative', 'Neutral', 'Positive', 'Label', 'Daily_Pct_Change']
#keep_cols is not in df_comb, so I need to fix this
df_merge = df_merge[keep_cols]

In [42]:
df_merge.dropna(subset=['Daily_Pct_Change'], inplace=True)

1988

In [ ]:
from datetime import datetime, timedelta

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#create and train the model
#this model gets the sentiment
# model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [ ]:
df_merge.head()

In [ ]:
#create the feature data set
X = df_merge
X = np.array(X.drop(['Label'], 1))
#create the target data set
y = np.array(df_merge['Label']) 

In [45]:
#need to define y_price and y_sentiment_label
#need to edit this code so that the sentiment values of the previous day are used to predict the stock price of the 
#next day
y_price = df_merge['Open']  # Assuming the column name for stock prices is 'Stock Price'
y_sentiment_label = df_merge['Label']

1988

In [54]:
#found input inconsisent with number of output -> drop outputs with nan values as well
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Assume you have prepared the combined feature vector 'X' and the target variables 'y_price' and 'y_sentiment_label'

# Split the data into training and test sets
# X_train, X_test, y_price_train, y_price_test, y_sentiment_train, y_sentiment_test = train_test_split(
#     X, y_price, y_sentiment_label, test_size=0.2, random_state=42
# )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Regression model for predicting stock price based on the sentiment values for from df_merge dataframe
regression_model = LinearRegression()
#THIS IS THE CURRENT LINE GIVING ME THE ERROR 
regression_model.fit(X_train, y_price) #need to clean with infinity, nan, and too big values for int64
price_predictions = regression_model.predict(X_test)

# Classification model for predicting sentiment label
classification_model = RandomForestClassifier()
classification_model.fit(X_train, y_sentiment_train)
sentiment_label_predictions = classification_model.predict(X_test)

# Now you have two arrays 'price_predictions' and 'sentiment_label_predictions'
# containing the predicted stock prices and sentiment labels, respectively

ValueError: Found input variables with inconsistent numbers of samples: [1590, 1988]

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
# Calculate the correlation between 'Daily_Pct_Change' and 'Positive'
correlation_positive = df_merge['Daily_Pct_Change'].corr(df_merge['Positive'])

# Calculate the correlation between 'Daily_Pct_Change' and 'Neutral'
correlation_neutral = df_merge['Daily_Pct_Change'].corr(df_merge['Neutral'])

# Calculate the correlation between 'Daily_Pct_Change' and 'Negative'
correlation_negative = df_merge['Daily_Pct_Change'].corr(df_merge['Negative'])

print(f"Correlation with Positive: {correlation_positive}")
print(f"Correlation with Neutral: {correlation_neutral}")
print(f"Correlation with Negative: {correlation_negative}")

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
y_train

In [ ]:
x_train #x_train is a numpy array

In [ ]:
type(x_test)

In [ ]:
#show the models predictions
predictions = model.predict(x_test)

In [ ]:
predictions[2]

In [ ]:
df_merge.head()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
mse = mean_squared_error(y_price_test, price_predictions)
mse

In [ ]:
#show the performance of the model
#0 = negative sentiment
#1 = positive sentiment
#price -> next value i need to implement
print(classification_report(y_test, predictions))

In [ ]:
#serialize into a file
import pickle

In [ ]:
with open('model_pickle', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open('model_pickle', 'rb') as f:
     mp = pickle.load(f)

In [ ]:
with open('model_price_pickle', 'wb') as f:
    pickle.dump(regression_model, f)

In [ ]:
with open('model_class_pickle', 'wb') as f:
    pickle.dump(classification_model, f)